In [81]:
import numpy as np
import pandas as pd
import statistics as stat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

In [82]:
df = pd.read_csv(r'C:\Users\Thomas Pettit\Desktop\MS&E 246\SBA_Loan_Data.csv', low_memory = False)

In [83]:
#DATA PREPROCESSING

# address missing values
df = df.drop(df.index[36780])
df = df.drop(df.index[102446])

#remove rows with canceled and exempt statuses
df = df.loc[df["LoanStatus"] != "CANCLD"]
df = df.loc[df["LoanStatus"] != "EXEMPT"]
df = df.reset_index(drop = True)

In [84]:
#ADDITIONAL VARIABLES


#Unemployment Rate @ origination
state_ur_col = []
state_ur_data = pd.read_csv(r'C:\Users\Thomas Pettit\Desktop\MS&E 246\State_Unemployment_Rates.csv')

for i in range(len(df)):
    state = df.at[i, "BorrState"]
    year = df.at[i, "ApprovalFiscalYear"]
    if (pd.isna(state) | pd.isna(year)):
        state_ur_col.append(np.nan)
    elif pd.isna(state_ur_data.at[year-1990, state]):
        state_ur_col.append(np.nanmean(state_ur_data[state]))
    else:
        u_rate = state_ur_data.at[year-1990, state]
        state_ur_col.append(u_rate)

df["State_UR"] = state_ur_col

In [85]:
# Add State GDP @ time of origination

state_gdp_data = pd.read_csv(r'C:\Users\Thomas Pettit\Desktop\MS&E 246\State_GDP_data.csv')
state_gdp_col = []

for i in range(len(df)):
    borr_state = df.at[i, "BorrState"]
    year_index = df.at[i, "ApprovalFiscalYear"] - 1997
    if borr_state == "GU":
        gdp_val = 4850
    elif borr_state == "VI":
        gdp_val = 3665
    elif year_index >= 0:
        gdp_val = state_gdp_data.at[year_index, borr_state]
    else:
        gdp_val = np.mean(state_gdp_data[borr_state])
        
    state_gdp_col.append(gdp_val)

df["State_GDP"] = state_gdp_col

In [86]:
#Add federal fund rate at approval year

fed_rate_data = pd.read_csv(r'C:\Users\Thomas Pettit\Desktop\MS&E 246\Fed_Fund_Rate.csv')
fed_rate_col = []

for i in range(len(df)):
    year = df.at[i, "ApprovalFiscalYear"]
    rate = fed_rate_data.iloc[year-1990, 0]
    fed_rate_col.append(rate)

df["Fed_Fund_at_Approval"] = fed_rate_col

In [87]:
#change to 2 digit NAICS codes
two_digit = []
for i in range(len(df["NaicsCode"])):
    elem = df.loc[i, "NaicsCode"]
    if (pd.isna(elem)):
        two_digit.append("Unknown")
    else:
        two_digit.append(str(elem)[:2])

df["2DigitNaics"] = two_digit

In [88]:
#Add S&P yearly return @ origination
sp500_df = pd.read_csv(r'C:\Users\Thomas Pettit\Desktop\MS&E 246\sp-500-historical-annual-returns.csv')
sp500 = []
for i in range(len(df)):
    year = df.loc[i, "ApprovalFiscalYear"]
    rtrn = sp500_df.iloc[year - 1990, 0]
    sp500.append(rtrn)

df["S&P_Return"] = sp500
    

In [89]:
#fill missing values
df["CDC_State"] = df["CDC_State"].fillna("Unknown")
df["ProjectState"] = df["ProjectState"].fillna("Unkmown")
df["BusinessType"] = df["BusinessType"].fillna("Unknown")
df["ThirdPartyDollars"] = df["ThirdPartyDollars"].fillna(0)

In [90]:
#Add total loan var
loan_sum = []
for i in range(len(df)):
    tot = 0
    thrd_pt = df.loc[i, "ThirdPartyDollars"]
    cdc_pt = df.loc[i, "GrossApproval"]
    tot = thrd_pt + cdc_pt
    loan_sum.append(tot)

df["LoanTotal"] = loan_sum

In [91]:
#create var checking relationship between borrower and proj/bank states
proj_borr_state_same = []
for i in range(len(df)):
    borr_state = df.at[i, "BorrState"]
    project_state = df.at[i, "ProjectState"]
    if borr_state == project_state:
        proj_borr_state_same.append(1)
    else:
        proj_borr_state_same.append(0)
        
borr_CDC_state_same = []
for i in range(len(df)):
    borr_state = df.at[i, "BorrState"]
    CDC_state = df.at[i, "CDC_State"]
    if borr_state == CDC_state:
        borr_CDC_state_same.append(1)
    else:
        borr_CDC_state_same.append(0)
        
#check if term multiple of year    
term_mult_year = []
for i in range(len(df)):
    months = df.loc[i, "TermInMonths"]
    if (months % 12 == 0):
        term_mult_year.append(1)
    else:
        term_mult_year.append(0)
        
df["ProjBorrState_Same"] = proj_borr_state_same
df["BorrCDCState_Same"] = borr_CDC_state_same
df["TermMultofYear"] = term_mult_year

In [92]:
#Encode Loan Status 
for i in range(len(df)):
    if (df.loc[i, "LoanStatus"] == "PIF"):
        df.at[i, "LoanStatus"] = 0
    elif (df.loc[i, "LoanStatus"] == "CHGOFF"):
        df.at[i, "LoanStatus"] = 1
        
df["LoanStatus"] = pd.to_numeric(df["LoanStatus"])

In [93]:
#check for NA's
for i in range(len(df.columns)):
    count = 0
    missing_index = []
    for j in range(len(df)):
        if pd.isna(df.iloc[j, i]):
            count +=1
            missing_index.append(j)            
    print(df.columns[i] + " has " + str(count) + " missing values at " + str(missing_index[0:10]) 
          + ".\n")

Program has 0 missing values at [].

BorrName has 0 missing values at [].

BorrStreet has 8 missing values at [2043, 2289, 4793, 6682, 10355, 16154, 16643, 17898].

BorrCity has 0 missing values at [].

BorrState has 0 missing values at [].

BorrZip has 0 missing values at [].

CDC_Name has 0 missing values at [].

CDC_Street has 192 missing values at [18, 90, 91, 113, 114, 167, 216, 217, 245, 314].

CDC_City has 192 missing values at [18, 90, 91, 113, 114, 167, 216, 217, 245, 314].

CDC_State has 0 missing values at [].

CDC_Zip has 192 missing values at [18, 90, 91, 113, 114, 167, 216, 217, 245, 314].

ThirdPartyLender_Name has 37853 missing values at [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].

ThirdPartyLender_City has 37853 missing values at [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].

ThirdPartyLender_State has 37738 missing values at [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].

ThirdPartyDollars has 0 missing values at [].

GrossApproval has 0 missing values at [].

ApprovalDate has 0 missing values at [].

Approv

In [94]:
dups = df[df.duplicated(['BorrName'])].loc[:, "BorrName"]
dups = set(dups)

duplicate = []
for i in range(len(df)):
    name = df.loc[i, "BorrName"]
    if (name in dups):
        duplicate.append(1)
    else:
        duplicate.append(0)

df["RepeatBorr"] = duplicate

In [95]:
#Normalize Loan_Total
mean_approval = stat.mean(df["LoanTotal"])
stdev_approval = stat.stdev(df["LoanTotal"])
df.loc[:, "LoanTotal_Norm"] = df.loc[:, "LoanTotal"].apply(lambda x: (x-mean_approval)/stdev_approval)

In [109]:
#Select regressor data
data_to_use = ["LoanTotal_Norm", "TermInMonths", "State_UR", "TermMultofYear", "BorrCDCState_Same",
              "ProjBorrState_Same", "S&P_Return", "BusinessType", "LoanStatus", "RepeatBorr",
              "subpgmdesc"]

In [110]:
df_to_use = df[data_to_use]
df_to_use = pd.get_dummies(df_to_use).astype(float)

#CORPORATION Status captures most of the Business type information
#subpgmdesc_Sec. 504 - Loan Guarantees - Private Sector Financed captures most subpgmdesc information
df_to_use = df_to_use.drop(["BusinessType_INDIVIDUAL", "BusinessType_PARTNERSHIP", "BusinessType_Unknown",
                           "subpgmdesc_504 Refinance", "subpgmdesc_Sec. 504 - Delta loans, funded 9/26/95",
                           "subpgmdesc_Sec. 504 - Premier Certified Lender Program"], axis = 1)



In [112]:
df_to_use.to_csv(r'C:\Users\Thomas Pettit\Desktop\MS&E 246\regression_SBA_data.csv', index = False)